# Submitting Notebooks to Braket Hybrid Jobs with Papermill

In this notebook, we submit a Jupyter notebook to Amazon Braket Hybrid Jobs. We download the resultant notebook. 

## Steps
1. First, we add the parameters tag to the top cell of our Jupyter notebook. This keeps all parameters in one place so they are easy to find. In this example, the parameter is `shots`. 
2. Add the `device_arn` and `results_dir` parameters to the same cell. These are Braket specific parameters that are needed to run the on the currect device with priority and to recover any files saved from the notebook.
3. Make sure the any files the notebook saves are prefixed with  `f"{results_dir}/`. For example, in the notebook `plt.savefig(f"{fig_dir}/histogram.png")`
4. In a new notebook, set the parameters using a Python dictionary. Do not include the the `device_arn` and `results_dir`. These are braket specifc parameters.
5. Create a job using the provded `runner.py` script and pointing to the source notebook. 
6. Download the results with the helper function `download_result_notebook(dir_name):` where `dir_name` is the name of the directory you want to save results to. 
7. Open the directory to see the completed notebook and any saved files. 



In [28]:
import os
from braket.aws import AwsQuantumJob
from braket.jobs.local import LocalQuantumJob

The first step is to remmeber all the hyperparamters we want to run the notebook with. 
For reference, we can look at the top cell of `notebook.ipynb` to find all hyperparamters. 

We specify the hyperparamters as a Python dictionary: 

In [29]:
hyperparameters = {"shots": 500}

In [30]:
device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

job = AwsQuantumJob.create(
    device=device_arn,
    source_module="src",
    entry_point="src.runner",
    hyperparameters=hyperparameters,
    input_data="src/0_Getting_started_papermill.ipynb"
)

Next we wait for the job to complete. When it finishes, the following cell with return `{}`.

In [31]:
job.result()

{}

Now we download the Jupyter notebook we ran within the job. First we make a directory to save the results to. 



In [45]:
import os
import tarfile
from boto3.session import Session

def download_result_notebook(dir_name="result"):
    PATH = f"{dir_name}"
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    path = job.metadata()["outputDataConfig"]["s3Path"] + "/output/model.tar.gz"
    bucket_name = path.split("/")[2]
    output_path = "/".join(path.split("/")[3:])

    session = Session()
    s3 = session.resource("s3")
    your_bucket = s3.Bucket(bucket_name)
    your_bucket.download_file(output_path, f"{dir_name}/model.tar.gz")

    with tarfile.open(f"{dir_name}/model.tar.gz") as file:
        file.extractall(f"./{dir_name}")

In [47]:
download_result_notebook("result3")

We first use AWS boto to find the correct directory, then use the boto S3 client to download the results files. 

Now there should be a file called `0_Getting_started_papermill.ipynb` in the result directory. This notebook was run with `hyperparameters` within the Braket job. 

We can also see that the `0_Getting_started_papermill.ipynb` saved the histogram figure as a PNG file. 